In [20]:
%matplotlib inline

In [17]:
import networkx as nx
import pandas as pd
from graph_helpers import load_graph_by_name
from collections import defaultdict
import metis

In [221]:
def get_cut(nodes):
    node_set = set(nodes)
    n_cut = 0
    for u in nodes:
        for v in g.neighbors(u):        
            if v not in node_set:
                n_cut += 1
    return n_cut       

In [222]:
def cut_from_each_node(nodes):
    node_set = set(nodes)
    cut = {}
    for u in nodes:
        cut[u] = 0
        for v in g.neighbors(u):        
            if v not in node_set:
                cut[u] += 1
    return cut       

In [247]:
gt = load_graph_by_name('grqc')

load graph from data/grqc/graph.gt


In [250]:
list(gt.vertex(0).out_neighbours())

[<Vertex object with index '3605' at 0x7f2935c57a50>]

In [204]:
g = nx.read_graphml('data/grqc/graph.graphml')


In [206]:
new_g = nx.Graph()
for u, v in g.edges_iter():
    new_g.add_edge(int(u), int(v))

In [213]:
g = new_g

In [256]:
assert gt.num_vertices() == g.number_of_nodes()
assert gt.num_edges() == g.number_of_edges()

for u, v in g.edges_iter():
    assert gt.edge(u, v) is not None


In [216]:
k = 50
cuts, comm = metis.part_graph(g, k)

In [217]:
comm_list = defaultdict(list)
for v, c in enumerate(comm):
    comm_list[c].append(v)

In [218]:
comm_cuts = {c: get_cut(nodes) for c, nodes in comm_list.items()}
records = [{'id': c, 'nodes': comm_list[c], 'cut': comm_cuts[c]} for c in comm_list]
df = pd.DataFrame.from_records(records)
df['size'] = df['nodes'].apply(len)
df['ratio'] = df['size'] / df['cut']

In [243]:
top_comms = df.sort_values('ratio', ascending=False).head(12)
top_comms.index

Int64Index([28, 43, 15, 8, 37, 30, 27, 26, 39, 33, 3, 11], dtype='int64')

In [246]:
for i, cid in enumerate(top_comms.index): 
    c = np.ones(g.number_of_nodes()) * -1
    nodes = df.iloc[cid]['nodes']
    cd = cut_from_each_node(nodes)
    source = min(nodes, key=cd.__getitem__)
    c[nodes] = 1    
    c[source] = 0
    
    for j in range(8):
        obs = np.random.permutation(nodes)[:int(len(nodes) * 0.2)]
        pkl.dump((obs, c), open('cascade-weighted/grqc-mcom-s0.02-o0.2/{}.pkl'.format(i * 8 + j), 'wb'))

In [257]:
import pickle as pkl
import numpy as np
from helpers import infected_nodes
cuts = []
for i in np.arange(0, 96, 8):
    _, c = pkl.load(open('cascade-weighted/grqc-msi-s0.02-o0.2/{}.pkl'.format(i), 'rb'))
    nodes = list(infected_nodes(c))
    cuts.append(get_cut(nodes))

In [258]:
print('mean cut', np.mean(cuts))

mean cut 481.3333333333333


In [260]:
print('mean cut in this case', top_comms['cut'].mean())

mean cut in this case 41.333333333333336


In [270]:
from glob import glob
def mean_runnig_time(d):
    return np.mean([pkl.load(open(p, 'rb'))['time_elapsed'] for p in glob(d + '/*.meta.pkl')])

In [271]:
mean_runnig_time('outputs/queries-weighted/grqc-msi-s0.02-o0.2/loop_erased/entropy')

269.7858134607474

In [273]:
mean_runnig_time('outputs/queries-weighted/grqc-mcomm-s0.02-o0.2/loop_erased/entropy')

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan